In [1]:
import datetime
import random

import numpy as np
from keras.layers import Activation
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dense
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, Model
from keras.applications.resnet50 import ResNet50
from keras.applications import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.wrappers.scikit_learn import KerasClassifier
from tqdm import tqdm
from sklearn.model_selection import GridSearchCV

from common import *

Using TensorFlow backend.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [9]:
print(".. setting base configuration & params ..")

initialize()

best_model_filepath = './saved-models/grid_search-01.hdf5'
images_size = 270


.. setting base configuration & params ..


In [10]:
(train_tensors, X_train, y_train,
test_tensors, X_test, y_test, 
valid_tensors, X_validate, y_validate, 
duration_loading) = load_and_split_data(images_size=images_size)

  1%|          | 17/1408 [00:00<00:08, 164.33it/s]

.. loading & splitting data ..


100%|██████████| 352/352 [00:03<00:00, 112.35it/s]


In [19]:
def create_model(optimizer, activation, network_layers, kernel_size):
    model = Sequential()
    
    filter_count = 16
    for layer_number in range(0, network_layers):

        if layer_number == 0:
            model.add(Conv2D(filters=filter_count, kernel_size=kernel_size, padding='same', 
                             input_shape=(270, 270, 3), data_format="channels_last"))
        else:
            model.add(Conv2D(filters=32, kernel_size=kernel_size, padding='same'))
            
        model.add(BatchNormalization())
        model.add(Activation(activation))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        
        filter_count = filter_count * 2

    model.add(GlobalAveragePooling2D())
    model.add(Dense(2, activation='softmax'))

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
model_wrapper = KerasClassifier(build_fn=create_model, verbose=0)

hyperparameters = dict(optimizer=['rmsprop', 'adam'], 
                       activation=['relu', 'sigmoid'],
                       network_layers=[2, 3, 4],                       
                       kernel_size=[2, 3],
                       epochs=[3], 
                       batch_size=[10, 50],                  
                      )


grid = GridSearchCV(estimator=model_wrapper, param_grid=hyperparameters, verbose=2)

grid_result = grid.fit(train_tensors, y_train,
              validation_data=(valid_tensors, y_validate),
              verbose=1)

print(grid_result.best_params_)


Fitting 3 folds for each of 48 candidates, totalling 144 fits
[CV] activation=relu, batch_size=10, epochs=5, kernel_size=2, network_layers=2, optimizer=rmsprop 
Train on 938 samples, validate on 352 samples
Epoch 1/5
938/938 [==============================] - 156s 166ms/step - loss: 0.4934 - acc: 0.7633 - val_loss: 0.4052 - val_acc: 0.8125
Epoch 2/5
938/938 [==============================] - 145s 155ms/step - loss: 0.4470 - acc: 0.7793 - val_loss: 0.3953 - val_acc: 0.8125
Epoch 3/5
280/938 [=======>......................] - ETA: 1:19 - loss: 0.4161 - acc: 0.8107

KeyboardInterrupt: 